In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import collections


In [2]:
path_dt = './data/full_house_again.csv'
df = pd.read_csv(path_dt)

In [3]:
df.head()

,Unnamed: 0,County_EN,YYYY_WW,BI,CI,HI,Temperature,T Max,T Min,RH,Precp
0,0,Hualien,2012_01,0.5,6.249750000000001,0.5,16.614286,18.657143,14.714286,75.714286,0.571429
1,1,Hualien,2012_02,0.0,0.0,0.0,18.942857,21.557143,16.800000,75.428571,0.357143
2,2,Hualien,2012_03,0.0,0.0,0.0,19.300000,21.914286,17.100000,76.428571,2.400000
3,3,Hualien,2012_04,XXX,XXX,XXX,16.142857,18.600000,13.771429,79.714286,5.000000
4,4,Hualien,2012_05,0.0,0.0,0.0,17.957143,20.414286,16.057143,76.571429,1.000000


In [4]:
X = ['Temperature', 'T Max', 'T Min', 'RH', 'Precp']
Y = ['BI', 'CI', 'HI']
YX = ['County_EN', 'YYYY_WW', 'BI', 'CI', 'HI', 'Temperature', 'T Max', 'T Min', 'RH', 'Precp']

In [8]:
yx = df[YX]
yx.head()

,County_EN,YYYY_WW,BI,CI,HI,Temperature,T Max,T Min,RH,Precp
0,Hualien,2012_01,0.5,6.249750000000001,0.5,16.614286,18.657143,14.714286,75.714286,0.571429
1,Hualien,2012_02,0.0,0.0,0.0,18.942857,21.557143,16.800000,75.428571,0.357143
2,Hualien,2012_03,0.0,0.0,0.0,19.300000,21.914286,17.100000,76.428571,2.400000
3,Hualien,2012_04,XXX,XXX,XXX,16.142857,18.600000,13.771429,79.714286,5.000000
4,Hualien,2012_05,0.0,0.0,0.0,17.957143,20.414286,16.057143,76.571429,1.000000


In [9]:
cty_yx = collections.defaultdict(list)

for i in range(yx.shape[0]):

    
    row = yx.iloc[i, :].tolist()
    
    cnty = row[0]
    
    cty_yx[cnty].append(row)
    

In [10]:
cty_yx_df = collections.defaultdict()

for k, v in cty_yx.items():

    x = pd.DataFrame(cty_yx[k], columns = YX)

    cty_yx_df[k] = x


In [11]:
cty_yx_df['Taipei'].head()

,County_EN,YYYY_WW,BI,CI,HI,Temperature,T Max,T Min,RH,Precp
0,Taipei,2012_01,1.3805873015873016,2.489971111111111,1.1681873015873017,14.500000,16.557143,13.028571,85.428571,10.200000
1,Taipei,2012_02,1.1770250915750915,2.9363352694924125,0.9870742804814234,17.542857,19.757143,16.257143,85.428571,1.000000
2,Taipei,2012_03,1.2979166666666666,1.20925,1.051125,17.685714,20.828571,15.857143,82.857143,4.385714
3,Taipei,2012_04,2.3205,12.091333333333333,0.9871666666666666,13.771429,15.914286,12.100000,89.571429,9.885714
4,Taipei,2012_05,1.7666666666666668,1.984,1.3666666666666667,15.400000,17.257143,13.785714,87.714286,3.000000


In [21]:
tp = cty_yx_df['Taipei'].iloc[2:5, :]
tp.reset_index()


,index,County_EN,YYYY_WW,BI,CI,HI,Temperature,T Max,T Min,RH,Precp
0,2,Taipei,2012_03,1.2979166666666666,1.20925,1.051125,17.685714,20.828571,15.857143,82.857143,4.385714
1,3,Taipei,2012_04,2.3205,12.091333333333333,0.9871666666666666,13.771429,15.914286,12.100000,89.571429,9.885714
2,4,Taipei,2012_05,1.7666666666666668,1.984,1.3666666666666667,15.400000,17.257143,13.785714,87.714286,3.000000


In [28]:
def lag_df(df, lag = 1, y_cols = ['County_EN', 'YYYY_WW', 'BI', 'CI', 'HI'], x_cols = ['Temperature', 'T Max', 'T Min', 'RH', 'Precp']):
    
    YY = df[y_cols]
    XX = df[x_cols]

    n_rows = YY.shape[0]
    
    yy = YY.iloc[lag:, :]
    yy = yy.reset_index()
    xx = XX.iloc[:-lag, :]
    xx = xx.reset_index()
    
    yyxx = pd.concat([yy, xx], axis = 1)
    
    return yyxx



In [33]:
lags = 5

for lag in range(1, lags, 1):
    
    print('lag = ', lag)
    
    tmp = []

    for cty in cty_yx_df.keys():
        
        df = cty_yx_df[cty]
        
        df2 = lag_df(df, lag = lag)
        
        tmp.append(df2)
        
    fn = pd.concat(tmp, axis = 0, ignore_index = True)

    fn.to_csv('./data/lag_%s_week_bi_temp.csv' %str(lag))

lag =  1
lag =  2
lag =  3
lag =  4
